In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from sklearn.metrics import accuracy_score

from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


from torch.utils.data import DataLoader
from torchvision import transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#### create path to my data on pc

In [3]:
train_path = r"C:\Users\1\Desktop\University_semeter_3\ZNEUS\archive\ASL_Dataset\Train"
test_path = r"C:\Users\1\Desktop\University_semeter_3\ZNEUS\archive\ASL_Dataset\Test"

#### create csv file with image pases and labels

In [4]:
data = []
test_data = []

for folder in os.listdir(train_path):
    folder_path = os.path.join(train_path, folder)

    if os.path.isdir(folder_path):
        for image in os.listdir(folder_path):
            if image.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image)
                data.append({'path': image_path, 'label': folder})

for folder in os.listdir(test_path):
    folder_path = os.path.join(test_path, folder)

    if os.path.isdir(folder_path):
        for image in os.listdir(folder_path):
            if image.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(folder_path, image)
                test_data.append({'path': image_path, 'label': folder})


# Створюємо DataFrame
df = pd.DataFrame(data)
df_test = pd.DataFrame(test_data)

# Зберігаємо в CSV
df.to_csv('dataset.csv', index=False)
df_test.to_csv('test_dataset.csv', index=False)
print(f" {len(df)} rows")
print(f"Clases: {df['label'].unique()}")


 165670 rows
Clases: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'Nothing' 'O' 'P'
 'Q' 'R' 'S' 'Space' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


In [5]:
ds = pd.read_csv('dataset.csv')
ds_test = pd.read_csv('test_dataset.csv')
ds.head()


,path,label
0,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
1,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
2,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
3,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A
4,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...,A


In [6]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165670 entries, 0 to 165669
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   path    165670 non-null  object
 1   label   165670 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [7]:
ds.duplicated().sum()

np.int64(0)

In [8]:
ds = ds.sample(frac=1, random_state=42).reset_index(drop=True)
ds_test = ds_test.sample(frac=1, random_state=42).reset_index(drop=True)

In [9]:
ds_answer =ds.iloc[:,-1:]
ds_test_answer = ds_test.iloc[:,-1:]
#ds_answer.head()
ds_test_answer.head()

,label
0,K
1,P
2,B
3,L
4,K


In [10]:
ds_train = ds.iloc[:,:-1]
ds_test_train = ds_test.iloc[:,:-1]
ds_train.head()

,path
0,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...
1,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...
2,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...
3,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...
4,C:\Users\1\Desktop\University_semeter_3\ZNEUS\...


In [13]:
# for i in range(3):
#     img = Image.open(ds_train.iloc[i,0])
#     img.show()




In [11]:
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

img_path = df.iloc[0]['path']
img = Image.open(img_path)

img_tensor = transform(img)

print(img_tensor.shape)

torch.Size([1, 128, 128])


In [12]:
# make dataset with tensors
class ASLDataset(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

        unique_labels = sorted(self.data['label'].unique())
        self.label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = Image.open(self.data.iloc[idx]['path'])
        label = self.data.iloc[idx]['label']
        label_idx = self.label_to_idx[label]
        return self.transform(img), label_idx


dataset = ASLDataset('dataset.csv', transform=transform)
testset = ASLDataset('test_dataset.csv', transform=transform)
loader = DataLoader(
    dataset,
    batch_size=128,
    num_workers=0,
    shuffle=True)

test_loader = DataLoader(
    testset,
    batch_size=128,
    num_workers=0,
    shuffle=True
)


In [13]:
conv1 = nn.Conv2d(1,6,3,1)
conv2 = nn.Conv2d(6,12,3,1)
conv3 = nn.Conv2d(12,18,3,1)
conv4 = nn.Conv2d(18,24,3,1)
conv5 = nn.Conv2d(24,32,3,1)
conv6 = nn.Conv2d(32,32,3,1)

In [14]:
x = F.max_pool2d(img_tensor.unsqueeze(0),2,2)
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 1, 32, 32])

In [66]:
x = F.relu(conv1(img_tensor.unsqueeze(0)))
x.shape

torch.Size([1, 6, 126, 126])

In [67]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 6, 63, 63])

In [68]:
x = F.relu(conv2(x))
x.shape

torch.Size([1, 12, 61, 61])

In [69]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 12, 30, 30])

In [70]:
x = F.relu(conv3(x))
x.shape

torch.Size([1, 18, 28, 28])

In [71]:
x = F.max_pool2d(x,2,2)
x.shape

torch.Size([1, 18, 14, 14])

In [14]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,6,3,1)
        self.conv2 = nn.Conv2d(6,12,3,1)
        self.conv3 = nn.Conv2d(12,18,3,1)
        self.conv4 = nn.Conv2d(18,24,3,1)
        self.conv5 = nn.Conv2d(24,32,3,1)
        self.fc1 = nn.Linear(18*14*14,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,29)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x,2,2)

        x = x.view(-1,18*14*14)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        return F.log_softmax(x, dim=1)

In [15]:
model = ConvolutionalNetwork().to(device)
model


ConvolutionalNetwork(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(12, 18, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(18, 24, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3528, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=29, bias=True)
)

In [16]:
import time
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
Number_of_epochs = 2
for epoch in range(Number_of_epochs):
    for i, (images, labels) in enumerate(loader):
        if i == 0 and epoch == 0:
            print(f"Images: {images.shape}, Labels: {labels.shape}")

        images = images.to(device)
        labels = labels.to(device)

        y_pred = model(images)
        loss = loss_func(y_pred, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/10], Step [{i+1}], Loss: {loss.item():.4f}')

Images: torch.Size([128, 1, 128, 128]), Labels: torch.Size([128])
Epoch [1/10], Step [1], Loss: 3.3663
Epoch [1/10], Step [101], Loss: 2.8663
Epoch [1/10], Step [201], Loss: 1.6219
Epoch [1/10], Step [301], Loss: 1.2488
Epoch [1/10], Step [401], Loss: 0.7789
Epoch [1/10], Step [501], Loss: 0.5587
Epoch [1/10], Step [601], Loss: 0.3925
Epoch [1/10], Step [701], Loss: 0.1934
Epoch [1/10], Step [801], Loss: 0.2819
Epoch [1/10], Step [901], Loss: 0.2382
Epoch [1/10], Step [1001], Loss: 0.2031
Epoch [1/10], Step [1101], Loss: 0.2424
Epoch [1/10], Step [1201], Loss: 0.0885
Epoch [2/10], Step [1], Loss: 0.1131
Epoch [2/10], Step [101], Loss: 0.1231
Epoch [2/10], Step [201], Loss: 0.0993
Epoch [2/10], Step [301], Loss: 0.0540
Epoch [2/10], Step [401], Loss: 0.0485
Epoch [2/10], Step [501], Loss: 0.0439
Epoch [2/10], Step [601], Loss: 0.1251
Epoch [2/10], Step [701], Loss: 0.0303
Epoch [2/10], Step [801], Loss: 0.0675
Epoch [2/10], Step [901], Loss: 0.0390
Epoch [2/10], Step [1001], Loss: 0.031

In [17]:
model.eval()


correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 96.43%
